<a href="https://colab.research.google.com/github/YerinS/Yerin_Portfolio/blob/main/DL_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [DL GAN] Handwritten Digits


- 　__Data__ 　　　　　　　MNIST dataset<br/>
- 　__Data Source__ 　 　　 TensorFlow Datasets collection<br/>
- 　__DL Framework__ 　 　 Keras<br/>
- 　__Packages__ 　 　 　　TensorFlow v2.7, NumPy, Matplotlib, IPython<br/>
<br/>

In [1]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
import imageio
import glob
from IPython import display
from matplotlib import pyplot as plt
import time

# 1. Prepare & Manipulate Data

- Download data from https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz

In [2]:
class DataReader():
    def __init__(self):
        (self.train_X, _), (_, _) = keras.datasets.mnist.load_data()
        self.train_X = self.preprocess(self.train_X)
        self.train_dataset = tf.data.Dataset.from_tensor_slices(self.train_X).shuffle(60000).batch(256)

    def preprocess(self, images):
        images = images.reshape(images.shape[0], 28, 28, 1).astype('float32')
        images = images / 127.5 - 1
        return images

    def show_processed_images(self):
        plt.figure(figsize=(10, 10))
        for i in range(25):
            plt.subplot(5, 5, i + 1)
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            plt.imshow(self.train_X[i])
        plt.show()

In [3]:
dr = DataReader()

11501568/11490434 [==============================] - 0s 0us/step


# 2. Make Generator and Discriminator

In [4]:
def make_generator():
    model = keras.Sequential([
        keras.layers.Dense(7*7*256, use_bias=False, input_shape=(100,)),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(),

        keras.layers.Reshape((7, 7, 256)),

        keras.layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(),

        keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(),

        keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])

    return model

In [5]:
def make_discriminator():
    model = keras.Sequential([
        keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]),
        keras.layers.LeakyReLU(),
        keras.layers.Dropout(0.3),

        keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        keras.layers.LeakyReLU(),
        keras.layers.Dropout(0.3),

        keras.layers.Flatten(),
        keras.layers.Dense(1)
    ])

    return model

In [6]:
def loss_G(fake_output):
    return keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output)

In [7]:
def loss_D(real_output, fake_output):
    real_loss = keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    fake_loss = keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [8]:
generator_optimizer = keras.optimizers.Adam(1e-4)
discriminator_optimizer = keras.optimizers.Adam(1e-4)

In [9]:
noise_dim = 100
seed = tf.random.normal([36, noise_dim])

In [10]:
@tf.function
def train_step(generator, discriminator, images):
    noise = tf.random.normal([256, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = loss_G(fake_output)
        disc_loss = loss_D(real_output, fake_output)

    gradient_G = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradient_D = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradient_G, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradient_D, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [11]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(6, 6))

    for i in range(predictions.shape[0]):
        plt.subplot(6, 6, i+1)
        plt.imshow(((predictions[i, :, :, 0]) + 1)/2)
        plt.axis('off')

    plt.savefig('results/image_at_epoch_{:04d}.png'.format(epoch))
    plt.close(fig)

# 3. Train Model

In [12]:
def train(generator, discriminator, dataset, epochs):
    if "results" not in os.listdir():
        os.mkdir("results")

    for epoch in range(epochs):
        start = time.time()
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(generator, discriminator, image_batch)
        duration = time.time() - start
        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch + 1, seed)
        print("Epoch " + str(epoch + 1) + "   Generator Loss : " + str(float(gen_loss))[:7]
                        + "   Discriminator Loss : " + str(float(disc_loss))[:7]
                        + "   Time : " + str(duration)[:5] + " seconds")

    display.clear_output(wait=True)
    generate_and_save_images(generator, epochs, seed)

In [13]:
generator = make_generator()
discriminator = make_discriminator()

train(generator, discriminator, dr.train_dataset, epochs=100)

Epoch 100   Generator Loss : 1.10146   Discriminator Loss : 1.16822   Time : 6.290 seconds


# 4. Save Result as a GIF

In [14]:
def gif_generation():
    anim_file = 'results/dcgan.gif'

    with imageio.get_writer(anim_file, mode='I') as writer:
        filenames = glob.glob('results/image*.png')
        filenames = sorted(filenames)
        last = -1
        for i, filename in enumerate(filenames):
            frame = 2 * (i ** 0.5)
            if round(frame) > round(last):
                last = frame
            else:
                continue
            image = imageio.imread(filename)
            writer.append_data(image)
        image = imageio.imread(filename)
        writer.append_data(image)

    import IPython
    if IPython.version_info > (6, 2, 0, ''):
        display.Image(filename=anim_file)

In [15]:
gif_generation()